In this demo, we use knowledge distillation to train a ResNet-18 model for image classification. We will show how to provide teacher model, student model, data loaders, inference pipeline and other arguments to the toolkit and start knowledge distillation training.

In [4]:
!pip install pytorch-lightning

     |████████████████████████████████| 829kB 11.2MB/s 
     |████████████████████████████████| 112kB 30.6MB/s 
     |████████████████████████████████| 829kB 24.6MB/s 
     |████████████████████████████████| 276kB 41.4MB/s 
     |████████████████████████████████| 1.3MB 35.5MB/s 
     |████████████████████████████████| 296kB 40.4MB/s 
     |████████████████████████████████| 143kB 40.3MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=06f9b891d5b8ddacaad64683fc5de0af17c6afb55d94ec557636541446145e3b
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=188d8596ad0ef13beabdf999cd2faa0e7cca3218b9edf169c931331d188bc343
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
  Found existing installation: future 0.16.0
  

## Download the toolkit 

In [1]:
!git clone https://github.com/georgian-io/Knowledge-Distillation-Toolkit.git

Cloning into 'Knowledge-Distillation-Toolkit'...
remote: Enumerating objects: 837, done.
remote: Counting objects: 100% (837/837), done.
remote: Compressing objects: 100% (671/671), done.
remote: Total 837 (delta 185), reused 771 (delta 140), pack-reused 0
Receiving objects: 100% (837/837), 3.31 MiB | 21.59 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [3]:
%cd Knowledge-Distillation-Toolkit/

/content/Knowledge-Distillation-Toolkit


In [4]:
import yaml
from collections import ChainMap

import torch
import torch.nn.functional as F
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision import datasets, transforms

from knowledge_distillation.kd_training import KnowledgeDistillationTraining

## Define the student model and teacher model

In [6]:
class StudentModel(ResNet):
    def __init__(self):
        super(StudentModel, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10) #ResNet18
        self.conv1 = torch.nn.Conv2d(1, 64,
            kernel_size=(7, 7),
            stride=(2, 2),
            padding=(3, 3), bias=False)

    def forward(self, batch, temperature=1):
        logits = super(StudentModel, self).forward(batch)
        logits = logits / temperature
        prob = F.softmax(logits, dim=0)
        log_prob = F.log_softmax(logits, dim=0)
        return {"logits":logits, "prob":prob, "log_prob":log_prob}

class TeacherModel(ResNet):
    def __init__(self):
        super(TeacherModel, self).__init__(BasicBlock, [3, 4, 6, 3], num_classes=10) #ResNet34
        self.conv1 = torch.nn.Conv2d(1, 64,
            kernel_size=(7, 7),
            stride=(2, 2),
            padding=(3, 3), bias=False)

    def forward(self, batch, temperature=1):
        logits = super(TeacherModel, self).forward(batch)
        logits = logits / temperature
        prob = F.softmax(logits, dim=0)
        log_prob = F.log_softmax(logits, dim=0)
        return {"logits":logits, "prob":prob, "log_prob":log_prob}

## Define the inference pipeline

In [7]:
class inference_pipeline:

    def __init__(self, device):
        self.device = device

    def run_inference_pipeline(self, model, data_loader):
        accuracy = 0
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(data_loader):
                X, y = data[0].to(self.device), data[1].to(self.device)
                outputs = model(X)
                predicted = torch.max(outputs["prob"], 1)[1]
                accuracy += predicted.eq(y.view_as(predicted)).sum().item()
        accuracy = accuracy / len(data_loader.dataset)
        return {"inference_result": accuracy}

In [8]:
def get_data_for_kd_training(batch):
    data = torch.cat([sample[0] for sample in batch], dim=0)
    data = data.unsqueeze(1)
    return data,

## Read from demo_config.yaml, which contains all argument set up

In [10]:
config = yaml.load(open('./examples/resnet_compression_demo/demo_config.yaml','r'), Loader=yaml.FullLoader)
device = torch.device("cuda")

## Create training and validation data loaders
We will use the MNIST dataset

In [17]:
# Create data loaders for training and validation
transform=transforms.Compose([
              transforms.ToTensor(),
              transforms.Normalize((0.1307,), (0.3081,))
              ])
train_kwargs = {'batch_size': 16, 'num_workers': 0}
test_kwargs = {'batch_size': 1000, 'num_workers': 0}
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)
train_data_loader = torch.utils.data.DataLoader(train_dataset, collate_fn=get_data_for_kd_training, **train_kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)
val_data_loaders = {"accuracy_on_validation_set": test_loader}


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Create an instance of inference pipeline

In [18]:
# Create inference pipeline for validating the student model
inference_pipeline_example = inference_pipeline(device)

## Create an instance of student model and teacher model

In [21]:
# Create student and teacher model
student_model = StudentModel()
teacher_model = TeacherModel()
teacher_model.load_state_dict(torch.load("./examples/resnet_compression_demo/trained_model/resnet34_teacher.pt"))

<All keys matched successfully>

## Pass data loaders, student and teacher model, inference pipeline and other argument set up into `KnowledgeDistillationTraining`

In [22]:
# Train a student model with knowledge distillation and get its performance on dev set
KD_resnet = KnowledgeDistillationTraining(train_data_loader = train_data_loader,
                                          val_data_loaders = val_data_loaders,
                                          inference_pipeline = inference_pipeline_example,
                                          student_model = student_model,
                                          teacher_model = teacher_model,
                                          num_gpu_used = config["knowledge_distillation"]["general"]["num_gpu_used"],
                                          final_loss_coeff_dict = config["knowledge_distillation"]["final_loss_coeff"],
                                          logging_param = ChainMap(config["knowledge_distillation"]["general"],
                                                                   config["knowledge_distillation"]["optimization"],
                                                                   config["knowledge_distillation"]["final_loss_coeff"],
                                                                   config["knowledge_distillation"]["pytorch_lightning_trainer"]),
                                          **ChainMap(config["knowledge_distillation"]["optimization"],
                                                     config["knowledge_distillation"]["pytorch_lightning_trainer"],
                                                     config["knowledge_distillation"]["comet_info"])
                                          )

Global seed set to 32
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


## Start knowledge distillation training

In [23]:
KD_resnet.start_kd_training()


  | Name          | Type         | Params
-----------------------------------------------
0 | student_model | StudentModel | 11.2 M
1 | teacher_model | TeacherModel | 21.3 M
-----------------------------------------------
32.5 M    Trainable params
0         Non-trainable params
32.5 M    Total params
129.836   Total estimated model params size (MB)



accuracy_on_validation_set :0.0666

GPU 0 current active MB: 131.81951999999998
GPU 0 current reserved MB: 157.2864



accuracy_on_validation_set :0.7412

GPU 0 current active MB: 266.921984
GPU 0 current reserved MB: 299.892736



accuracy_on_validation_set :0.8167

GPU 0 current active MB: 266.923008
GPU 0 current reserved MB: 297.79558399999996



accuracy_on_validation_set :0.8405

GPU 0 current active MB: 266.923008
GPU 0 current reserved MB: 299.892736



accuracy_on_validation_set :0.8569

GPU 0 current active MB: 266.923008
GPU 0 current reserved MB: 297.79558399999996



accuracy_on_validation_set :0.8696

GPU 0 current active MB: 266.923008
GPU 0 current reserved MB: 297.79558399999996



As the above output shows, validation accuracy of the student model improves in every training epoch